# ISYE/Math/CS 425 - Final - Helper
- Bryan Luu

All code in Julia 1.5.2

## Contents
- [Helper Functions](#Helper-Functions)
- [Max-Flow](#Max-Flow)
- [Optimal Matchings](#Optimal-Matchings)
- [TSP](#TSP)

Import libraries

In [1]:
using LightGraphs, MetaGraphs, LightGraphsMatching, GraphRecipes, Plots

## Helper Functions

In [2]:
# connects adjacent vertices in list of vertices P
connect(P::Vector{Int}, Etype=Edge) = [Etype(u, v) for (u, v) in zip(P[1:end-1], P[2:end])];
# checks if vertex v is incident to edge e
incident(v::Int, e::T) where {T <: AbstractEdge} = e.src == v || e.dst == v; 
# prints a sum: x[1] + x[2] + ... = sum(x)
sum_str(x::Vector{N}) where {N <: Number} = join(x, " + ") * " = $(sum(x))";
 # get the weight list of the edgelist from the weight dictionary
wts(E::Vector{T}, 
    wd::Dict{Tuple{Int, Int}, N}) where {T <: AbstractEdge, N <: Number} = [wd[(e.src, e.dst)] for e in E];
 # function to convert array into array of symbols
to_sym(v::Array) = map(x->Symbol(x), v);

# E list of Edges in a path, topologically sorted (i.e. E[1] -> E[2] -> ... -> E[end])
function disconnect(E::Vector{T}) where {T <: AbstractEdge}
    path = []
    s = nothing;
    for e in E
        s, d = e.src, e.dst;
        push!(path, s);
        s = d;
    end
    push!(path, s);
    return path
end

# Similar to Base.unique except includes options for undirected edges
function unique(E::Vector{T}; directed=false) where {T <: AbstractEdge}
    tmp = copy(E);
    new = T[];
    while !isempty(tmp)
        e = pop!(tmp);
        removed = remove!(e, tmp; directed=directed);
        push!(new, e);
    end
    return new;
end

# Returns a normalized x where (min(x), max(x)) -> (0, 1)
function normalize(x::N) where {N <: Union{Real, Array{<:Real}}}
    mx, Mx = min(x...), max(x...); # limits
    return Mx == mx ? x.*0 : (x.-mx)./(Mx .- mx)
end

# Set the position metadata of vertex v
function place!(mg::G, v::Int, 
        position::Tuple{T, N}) where {G <: Union{MetaGraph, MetaDiGraph}, T <: Real, N <: Real}
    set_prop!(mg, v, :position, position);
end

# Add edge between vertex u, v in MetaGraph mg, with weight metadata,
# updating the weight dictionary if given.
function connect!(mg::G, u::Int, v::Int, weight::N; 
        wd::D=nothing) where {G <: Union{MetaGraph, MetaDiGraph}, N <: Real, D <: Union{Nothing, Dict}}
    add_edge!(mg, u, v);
    set_prop!(mg, u, v, :weight, weight);
    if wd != nothing
        wd[(u, v)] = wd[(v, u)] = weight;
    end
end

# Remove edge e from a copy of edge list E, with a default flag for undirected edges
function remove(e::T, E::Vector{T}; directed::Bool=false, 
        condition=(k)->(k == e || (!directed && k == reverse(e)))) where {T <: AbstractEdge}
    return [k for k in E if !condition(k)];
end

# Remove edge e from edge list E in place, with a default flag for undirected edges
# Returns any edges removed
function remove!(e::T, E::Vector{T}; directed::Bool=false, 
        condition=(k)->(k == e || (!directed && k == reverse(e)))) where {T <: AbstractEdge}
    i = 1;
    removed = T[]
    while i <= length(E)
        if condition(E[i])
            push!(removed, popat!(E, i));
            continue;
        end
        i += 1;
    end
    return removed;
end

# Returns an edge-weight function that emphasizes the edges in edgelist E,
function emphasize(E::Vector{T}; in_weight=3, out_weight=1, directed=false) where {T <: AbstractEdge}
    function ew(s, d, w)
        e = Edge(s, d);
        re = reverse(e);
        if in(e, E) || (!directed && in(re, E))
            return in_weight;
        else
            return out_weight;
        end
    end
    return ew;
end;

# IJulia only... inline display
function show_plot(plt; show=true, title="")
    show || return; # return if show=false
    title!(plt, title);
    display(Main.IJulia.InlineDisplay(), plt);
end

# Helper function to plot the given metagraph with fixed positions
function plot_fixed(g::G; names=vertices(g), positions=[], 
        ew=(s, d, w)->1, args...) where {G <: Union{MetaGraph, MetaDiGraph}}
    N = nv(g);

    mg = copy(g); # create internal connected copy
    fake = Dict();
    if !is_connected(mg)
        for v in 2:N
            if is_connected(mg)
                break
            end
            add_edge!(mg, 1, v);
            fake[1, v] = true;
        end
    end

    new_ew(s, d, w) = get(fake, (s, d), false) ? 0 : ew(s, d, w);

    if isempty(positions)
        positions = [get_prop(g, v, :position) for v in 1:N];
    end
    
    x = [pos[1] for pos in positions];
    y = [pos[2] for pos in positions]; # Flip the y so that (0,0) is bottom-left.
    
    p = hcat(x, y); # Position
    sp = normalize(p); # Scale so that the positions are normalized
    
    graphplot(mg; x=sp[:, 1], y=sp[:, 2], names=names, ew=new_ew, args...);
end;

In [3]:
# Bread-first-search for (u,v)-path in the graph defined by graph g
function BFS(g::G, u::Int, v::Int) where {G <: AbstractGraph}
    p = Dict{Int, Int}(zip(vertices(g), -1*ones(nv(g))));
    p[u] = 0;
    q = Int[u]; # queue for BFS
    while !isempty(q)
        curr = popfirst!(q);
        if curr == v
            break
        end
        for n in neighbors(g, curr)
            if p[n] == -1
                push!(q, n);
                p[n] = curr;
            end
        end
    end
    i = v;
    path = Int[];
    while(i != u)
        if p[i] == -1
            return Int[]
        else
            pushfirst!(path, i);
            i = p[i];
        end
    end
    pushfirst!(path, u);
    return path
end;

In [4]:
function cut(g::AbstractGraph, U::Union{Set{Int}, Vector{Int}}; directed::Bool=is_directed(g))
    delta = Edge[];
    inside = Int[];
    outside = Int[];
    for e in edges(g)
        if in(e.src, U) && !in(e.dst, U)
            push!(delta, e);
            push!(inside, e.src);
            push!(outside, e.dst);
        elseif !directed && !in(e.src, U) && in(e.dst, U)
            push!(delta, e);
            push!(inside, e.dst);
            push!(outside, e.src);
        end
    end
    return delta, inside, outside;
end;

## Max Flow
- [Max-Flow Functions](#Max-Flow-Functions)
- [Max-Flow Input](#Max-Flow-Input)
- [Bi-partite Max Matchings Input](#Bi-partite-Max-Matchings-Input)

### Max-Flow Functions

In [ ]:
# calculate the flow into vertex v given flows x
function flow(x::Dict{T, N}, v::Int) where {T <: Union{AbstractEdge, Tuple{Int, Int}}, N <: Real}
    E = keys(x);
    inflow = sum(N[x[e] for e in E if e.dst==v]);
    outflow = sum(N[x[e] for e in E if e.src==v]);
    inflow - outflow
end;

In [ ]:
# Construct auxiliary graph of metagraph from flow x
function auxiliary_graph(mg::MetaDiGraph, x::Dict{T, N}) where {T <: AbstractEdge, N <: Real}
    aux = copy(mg);
    # Construct capacities
    u = Dict(e => get_prop(mg, e, :weight) for e in edges(mg));
    for e in edges(mg)
        rem_edge!(aux, e); # Delete the existing edge first
        if x[e] < u[e]
            add_edge!(aux, e); # add forward arc
            set_prop!(aux, e, :forward, true); # set edge to a forward arc
        end
        if x[e] > 0
            rev = reverse(e);
            add_edge!(aux, rev); # add a reverse arc
            set_prop!(aux, rev, :reverse, true); # set edge as reverse arc
        end
    end
    return aux;
end;

In [ ]:
# Augmenting path algorithm for metagraph mg with capacities defined by edge weights, and source and sink.
function APA(mg::MetaDiGraph, source, sink; names=Vector(1:nv(mg)), 
        verbose::Bool=false, show_graphs::Bool=false) where {T <: Union{String, Symbol, Int}}
    log(msg::Any) = verbose ? println(msg) : nothing # printer
    isreverse(aux::MetaDiGraph, e::T) where {T <: AbstractEdge} = has_prop(aux, e, :reverse);
    isforward(aux::MetaDiGraph, e::T) where {T <: AbstractEdge} = has_prop(aux, e, :forward);
    ename(e::T) where {T <: AbstractEdge} = "$(names[e.src]), $(names[e.dst])";
    index = Dict(name => i for (i, name) in enumerate(names));
    r, s = index[source], index[sink];
    
    # Construct capacities
    u = Dict(e => get_prop(mg, e, :weight) for e in edges(mg));
    u_max = max(values(u)...) # max capacity
    
    # Construct flow vector
    x = Dict{edgetype(mg), valtype(u)}(zip(edges(mg), zeros(ne(mg))));
    
    # Returns a scaled flow where x=0 -> low and x=max_flow -> high
    function flow_ew(low::T, high::N; max_flow=u_max) where {T <: Real, N <: Real}
        if max_flow == Inf
            return (s, d, w) -> high;
        end
        return (s, d, w) -> low + (max_flow == 0 ? 0 : (high-low)*x[Edge(s, d)]/max_flow);
    end
    plot_flow(g::MetaDiGraph; ew=flow_ew(0.5, 3), 
        args...) = plot_fixed(g; ew=ew, names=names, curvature=0.005, args...);
    
    log("---Setup---")
    log("Edge\tu, x")
    for e in edges(mg)
        log(ename(e) * ":\t$(u[e]), $(x[e])")
    end
    show_plot(plot_flow(mg; ew=(s, d, w)->1, el=Dict((e.src, e.dst)=>"$(u[e])" for e in edges(mg)), 
            elb=false, elo=0.01); show=show_graphs, title="Input graph");
    
    # Construct auxiliary digraph
    aux = auxiliary_graph(mg, x);
    
    path = BFS(aux, r, s);
    arcs = connect(path);
    i = 1;
    while(!isempty(path))
        log("---Iteration $(i)---")
        show_plot(plot_flow(aux; ew=emphasize(arcs; directed=true));
            show=show_graphs, title="Auxiliary graph at iteration $(i)");
        log("x-augmenting path: " * join(map((v)->string(names[v]), path), ", "))

        # find x-width
        eps1 = min(Inf, [u[e] - x[e] for e in arcs if isforward(aux, e)]...);
        eps2 = min(Inf, [x[reverse(e)] for e in arcs if isreverse(aux, e)]...);
        log("ε1: $(eps1)")
        log("ε2: $(eps1)")
        x_width = min(eps1, eps2);
        
        log("x-width: $(x_width)")

        if x_width == Inf
            break
        end

        for e in arcs
            if isforward(aux, e)
                x[e] += x_width; # increase flow on forward arcs
            end
            if isreverse(aux, e)
                x[reverse(e)] -= x_width; # decrease flow on reverse arcs
            end
        end

        log("Edge\tu, x")
        for e in edges(mg)
            log(ename(e) * ":\t$(u[e]), $(x[e])")
        end
        
        max_flow = (u_max == Inf ? min([u[e] for e in arcs]...) : u_max);
        show_plot(plot_flow(mg; ew=flow_ew(0.5, 3; max_flow=max_flow), 
                el=Dict((e.src, e.dst)=>"$(u[e]), $(x[e])" for e in edges(mg)), 
                elo=0.025, elb=false); show=show_graphs, title="Flow at iteration $(i)");

        # Create auxiliary graph from flows
        aux = auxiliary_graph(mg, x);
        log("Auxiliary graph: ")
        for e in edges(aux)
            log("$(names[e.src]) => $(names[e.dst])")
        end
        
        path = BFS(aux, r, s);
        if !isempty(path)
            arcs = connect(path);
        end
        i += 1;
    end

    show_plot(plot_flow(aux; ew=emphasize(arcs; directed=true)); show=show_graphs, title="Final auxiliary graph");
    log("No augmenting path left...")
    
    max_flow = flow(x, s);
                           
    log("===== OPTIMUM FOUND =====")
    log("Max-flow: $(max_flow)")
    show_plot(plot_flow(mg, el=Dict((e.src, e.dst)=>"$(u[e]), $(x[e])" for e in edges(mg)), 
                elo=0.025, elb=false); show=show_graphs, title="Optimal Flow");
                                    
    return (max_flow, x);
end;

In [ ]:
# Get maximum matching for bipartite graph defined by node sets P, Q, and adjacency list L
function max_matching_bipartite(P::Union{Set, Array}, Q::Union{Set, Array}, L::Dict;
        verbose::Bool=false, show_graphs::Bool=false, args...)
    log(msg::Any) = verbose ? println(msg) : nothing # printer
    # Return a centered array of num_elements numbers separated by spacing
    centered(num_elements::Int, 
        spacing::Real) = [i*spacing for i in 0:num_elements-1] .- (num_elements-1)*spacing/2;
    P, Q = collect(Base.unique(P)), collect(Base.unique(Q)); # Convert into arrays
    
    (isempty(P) || isempty(Q)) && return []; # return if P or Q is empty
    
    names = [:source, :sink, P..., Q...];
    index = Dict(name => i for (i, name) in enumerate(names));
    V = collect(1:length(names));
    r, s = index[:source], index[:sink]; # Indices of source and sink nodes
    wd = Dict(); # Weight dictionary

    # Build input graph for APA
    mg = MetaDiGraph(length(names));
    for p in P
        connect!(mg, r, index[p], 1; wd=wd); # Connect source to P nodes with weight 1
    end
    for q in Q
        connect!(mg, index[q], s, 1; wd=wd); # Connect Q to sink nodes with weight 1
    end
    for u in keys(L) # Connect P to Q nodes with weight Inf using adjacency list
        if in(u, P)
            for v in L[u]
                in(v, Q) && connect!(mg, index[u], index[v], Inf; wd=wd);
            end
        elseif in(u, Q)
            for v in L[u]
                in(v, P) && connect!(mg, index[v], index[u], Inf; wd=wd);
            end
        end
    end
    
    max_len = max(length(P), length(Q));
    vertical_spacing = 1;
    horizontal_spacing = vertical_spacing*(max_len - 1)/2;
    x_P = zeros(length(P));
    x_Q = horizontal_spacing*ones(length(Q));
    y_P = centered(length(P), vertical_spacing);
    y_Q = centered(length(Q), vertical_spacing);
    x_r = -horizontal_spacing;
    x_s = 2*horizontal_spacing;
    y_r = y_s = 0;
    
    place!(mg, r, (x_r, y_r));
    place!(mg, s, (x_s, y_s));
    for (i, p) in enumerate(P)
        place!(mg, index[p], (x_P[i], y_P[i]));
    end
    for (i, q) in enumerate(Q)
        place!(mg, index[q], (x_Q[i], y_Q[i]));
    end
    
#     input = MetaGraph(mg);
    V_PQ = filter((v)->(v != r && v != s), V); # node indices of P, Q
    ig, Imap = induced_subgraph(MetaGraph(mg), V_PQ); # graph induced by P, Q
    inv_Imap = Dict(v => i for (i, v) in enumerate(Imap)); # inverse Imap
    
    show_plot(plot_fixed(ig; names=[names[Imap[i]] for i in vertices(ig)], curvature=0, args...);
        show=show_graphs, title="Input graph");
    
    log("Using APA to solve for max-matchings...");
    
    max_matchings, x = APA(mg, r, s);
    
    E = [e for e in edges(mg)];
    M = [Edge(inv_Imap[e.src], inv_Imap[e.dst]) for e in edges(mg) if (e.src != r && e.dst != s) && x[e] > 0];
    Msize = length(M);
    
    show_plot(plot_fixed(mg; names=[((v == r || v == s) ? " " : names[v]) for v in V],
        ew=emphasize(filter((e)->(x[e] > 0), E), in_weight=2, out_weight=0.5), 
        curvature=0, args...); show=show_graphs, title="APA output flow");
    
    log("Maximum Matching M:");
    for e in M
        s, d = Imap[e.src], Imap[e.dst];
        log("$(names[s]) <=> $(names[d])");
    end
    log("|M| = $(Msize)");
    
    show_plot(plot_fixed(ig; names=[names[Imap[i]] for i in vertices(ig)], curvature=0,
        ew=emphasize(M; out_weight=0.5), args...);
        show=show_graphs, title="Maximum Matching M");
    
    matches = [(names[Imap[e.src]], names[Imap[e.dst]]) for e in M];
    return Msize, matches
end;

### Max-Flow Input

In [ ]:
V_str = "rpbacsqd"; # String representing all nodes
v_names = to_sym(collect(V_str)); # Node names
V = Vector(1:length(V_str)); # Node indices
index = Dict(v_names[i] => i for i in V); # Map from names to indices

L = Dict{Symbol, Any}(zip(v_names, [[] for v in V])); # Adjacency list
L[:r] = [:p, :a, :q];
L[:p] = [:q, :b];
L[:b] = [:a, :s];
L[:a] = [:c, :d];
L[:c] = [:q, :s, :b];
L[:q] = [:p, :b, :d];
L[:d] = [:c, :s];

Lu = Dict{Symbol, Any}(zip(v_names, [[] for v in V])); # Capacities in Adjacency List form
Lu[:r] = [6, 9, 4];
Lu[:p] = [2, 3];
Lu[:b] = [1, 8];
Lu[:a] = [8, 1];
Lu[:c] = [1, 4, 2];
Lu[:q] = [1, 2, 6];
Lu[:d] = [1, 6];

# Create metagraph
mg = MetaDiGraph(length(V_str));
for i in v_names
    for (k, j) in enumerate(L[i])
        s, d = index[i], index[j];
        add_edge!(mg, s, d);
        set_prop!(mg, s, d, :weight, Lu[i][k]);
    end
end

# Position vertices
place!(mg, index[:r], (0, 0));
place!(mg, index[:p], (1, 2));
place!(mg, index[:q], (2, -2));
place!(mg, index[:a], (2.3, 0.5));
place!(mg, index[:c], (3.5, 0));
place!(mg, index[:b], (3.6, 2));
place!(mg, index[:d], (4.4, -2));
place!(mg, index[:s], (5.5, 0));

In [ ]:
max_flow, x = APA(mg, :r, :s, names=v_names, verbose=false, show_graphs=false);
println("Max-flow: $(max_flow)");

### Bi-partite Max Matchings Input

In [ ]:
P_str = "abcdefghij"; # Specify P here as a string
Q_str = uppercase(P_str); # Specify Q here as a string

P, Q = to_sym(collect(P_str)), to_sym(collect(Q_str)); # creates arrays for P and Q

L = Dict();
L[:a] = [:A, :C, :D];
L[:b] = [:A, :B, :H];
L[:c] = [:A, :C];
L[:d] = [:C, :D];
L[:e] = [:C, :D];
L[:f] = [:B, :E, :G];
L[:g] = [:E, :F, :J];
L[:h] = [:E, :F, :G, :H, :J];
L[:i] = [:I];
L[:j] = [:D, :I];

Msize, M = max_matching_bipartite(P, Q, L; verbose=false, show_graphs=false, nodesize=0.25);
println("Max |M| = $(Msize)")

## Optimal Matchings
- [Bi-partite Max Matchings Input](#Bi-partite-Max-Matchings-Input)

In [5]:
struct UEdge{T} <: AbstractEdge{T}
    src::T
    dst::T
end

UEdge(s::T, d::T) where T = UEdge{T}(s, d);
UEdge(e::Edge) = UEdge(e.src, e.dst);

Edge(ue::UEdge) = Edge(ue.src, ue.dst);


Base.:(==)(e1::UEdge, 
    e2::UEdge) = (e1.src == e2.src && e1.dst == e2.dst) || (e1.src == e2.dst && e1.dst == e2.src);
Base.isequal(e1::UEdge, 
    e2::UEdge) = (e1 == e2);
Base.hash(e::UEdge, h::UInt) = Base.hash(Set([Symbol(e.src), Symbol(e.dst)]));
Base.print(io::IO, ue::UEdge{T}) where T = Base.print(io, "$(ue.src)-$(ue.dst)");
Base.print(io::IO, M::Set{UEdge{T}}) where T = Base.print(io, "{" * join(M, ", ") * "}");

In [6]:
mutable struct AltTree
    root::Int
    A::Set{Int}
    B::Set{Int}
    M::Set{UEdge{Int}}
    graph::SimpleGraph
    function AltTree(g::AbstractGraph, M::Set{UEdge{Int}})
        M_covered = union(Set(e.src for e in M), Set(e.dst for e in M));
        M_exposed = setdiff(Set(v for v in vertices(g)), M_covered);
        if isempty(M_exposed)
            error("Cannot build alternating tree with M-covered root")
        end
        r = first(M_exposed);
        new(r, Set{Int}(), Set{Int}([r]), M, SimpleGraph(nv(g)));
    end
    function AltTree(r::Int, A::Set{Int}, B::Set{Int}, M::Set{UEdge{Int}}, graph::SimpleGraph)
        new(r, A, B, M, graph);
    end
end;

In [ ]:
g = SimpleDiGraph(3)
add_edge!(g, 1, 2)
add_edge!(g, 2, 3)
cut(g, [1])
graphplot(g; names=vertices(g), nodecolor=[:red, RGB(0, 1, 0), :blue],
    nodeshape=[:circle, :rect, :hexagon], nodeweights=[1, 7, 1], edgecolor=(s, d, w)->(s == 1 ? :green : :black))

In [ ]:
# function extend!(T::AltTree, v::Int, w::Int)
#     M_covered = [e for e in T.M if incident(w, e)];
#     if in(v, T.B) && !in(w, union(T.A, T.B)) && !isempty(M_covered)
#         e = first(M_covered);
#         z = (w == e.src ? e.dst : e.src);
#         push!(T.A, w);
#         push!(T.B, z);
#         return (add_edge!(T.graph, v, w) && add_edge!(T.graph, w, z));
#     else
#         return false;
#     end
# end;

# function augment!(T::AltTree, v::Int, w::Int)
#     M_covered = [e for e in T.M if incident(w, e)];
#     if in(v, T.B) && !in(w, union(T.A, T.B)) && isempty(M_covered)
#         nodes = BFS(T.graph, T.root, v);
#         P = connect(nodes, UEdge{Int});
#         push!(P, UEdge(v, w));
#         symdiff!(T.M, P);
#         return true;
#     else
#         return false;
#     end
# end;

# function shrink(G::AbstractGraph, T::AltTree, v::Int, w::Int, S::Dict{Int, Set{Int}}=Dict{Int, Set{Int}}())
#     if !in(v, T.B) || !in(w, T.B) || !has_edge(G, v, w)
#         return false;
#     end
    
#     # Get circuit C
#     C = BFS(T.graph, v, w);
#     add_edge!(T.graph, v, w);
#     C_edges = Set(connect([C..., v], UEdge{Int}));
#     V_C = Set(C);
    
#     # Create new graph G' = G x C
#     V_GP = [v for v in vertices(G) if !in(v, C)];
#     Gnew, Gmap = induced_subgraph(G, V_GP);
#     add_vertex!(Gnew);
#     pseudo = nv(Gnew); # Create the pseudo node
    
#     # Generate maps
#     S = Dict(i => Set([v]) for (i, v) in enumerate(Gmap)); # Derived to original map
#     S[pseudo] = V_C; # Pseudonode map
#     rev_S = Dict{Int, Int}(); # Original to derived map
#     for (i, s) in S
#         for v in s
#             rev_S[v] = i;
#         end
#     end
    
#     # Create new tree T' with E(T') = E(T) \ E(C)
#     V_T = union(T.A, T.B);
#     Tnew_graph, Tmap = induced_subgraph(T.graph, [v for v in V_T if rev_S[v] != pseudo]);
#     add_vertex!(Tnew_graph);
#     Tnew_A = Set([rev_S[v] for v in T.A]); # Convert original A to derived A
#     Tnew_B = Set([rev_S[v] for v in T.B]); # Convert original B to derived B
#     Tnew_root = rev_S[T.root];
    
#     # Copy M into Mnew and map to derived vertices
#     Mnew = Set([UEdge(rev_S[e.src], rev_S[e.dst]) for e in M]);
#     # Remove matches in C from M
#     pop!(Mnew, UEdge(pseudo, pseudo));
    
#     edges_to_C, C_inside, C_neighbors = cut(G, C);
#     for (i, e) in enumerate(edges_to_C)
#         n = C_neighbors[i]; # Get neighbor of C
#         n_derived = first(rev_S[n]); # Derived index of neighbor
#         add_edge!(Gnew, n_derived, pseudo); # Connect neighbor to the new pseudonode's index
#         if has_edge(T.graph, e)
#             add_edge!(Tnew_graph, n_derived, pseudo); # Connect neighbor edge in Tnew
#         end
#     end
    
#     # Mnew = M \ E(C)
    
# #     # Modify alternating tree
# #     T.A = Tnew_A;
# #     T.B = Tnew_B;
# #     T.root = Tnew_root;
# #     T.graph = Tnew_graph;
#     Tnew = AltTree(Tnew_root, Tnew_A, Tnew_B, Mnew, Tnew_graph);
    
#     if isa(Gnew, MetaGraph) || isa(Gnew, MetaDiGraph)
#         positions = [get_prop(G, v, :position) for v in V_C];
#         centroid = (sum(p[1] for p in positions)/length(V_C), sum(p[2] for p in positions)/length(V_C));
#         place!(Gnew, pseudo, centroid); # Set pseudo node position to centroid
#     end
    
#     return Gnew, Mnew, Tnew, S, rev_S, pseudo;
# end;

# # Prints a nice representation of an alternating tree
# function Base.print(io::IO, T::AltTree)
#     Base.print(io, "{root: $(T.root); A: {" * join(T.A, ", "), "}; B: {" * join(T.B, ", ") * "}; "
#         * "E(T): $(Set(UEdge(e) for e in edges(T.graph)))}");
# end;

In [7]:
function extend!(T::AltTree, v::Int, w::Int)
    M_covered = [e for e in T.M if incident(w, e)];
    if in(v, T.B) && !in(w, union(T.A, T.B)) && !isempty(M_covered)
        e = first(M_covered);
        z = (w == e.src ? e.dst : e.src);
        push!(T.A, w);
        push!(T.B, z);
        return (add_edge!(T.graph, v, w) && add_edge!(T.graph, w, z));
    else
        return false;
    end
end;

function augment!(T::AltTree, v::Int, w::Int)
    M_covered = [e for e in T.M if incident(w, e)];
    if in(v, T.B) && !in(w, union(T.A, T.B)) && isempty(M_covered)
        nodes = BFS(T.graph, T.root, v);
        P = connect(nodes, UEdge{Int});
        push!(P, UEdge(v, w));
        symdiff!(T.M, P);
        return true;
    else
        return false;
    end
end;

function shrink(G::N, T::AltTree, v::Int, w::Int,
        S::Dict{Int, Set{Int}}=Dict{Int, Set{Int}}()) where {N <: AbstractGraph}
    if !in(v, T.B) || !in(w, T.B) || !has_edge(G, v, w)
        return false;
    end
    
    println("given M: $(M)")
    
    # Get circuit C
    C_path = BFS(T.graph, v, w);
    add_edge!(T.graph, v, w);
    C = connect([C_path..., v], UEdge{Int});
    C_edges = Set(C);
    V_C = Set(C_path);
    
    # Create new graph G' = G x C
    V_GP = [v for v in vertices(G) if !in(v, V_C)];
    Gnew, Gmap = induced_subgraph(G, V_GP);
    add_vertex!(Gnew);
    pseudo = nv(Gnew); # Create the pseudo node
    
    if isempty(S)
        S = Dict(v => Set([v]) for v in vertices(G));
    end
    
    # Generate maps
#     s = Dict(i => Set([v]) for (i, v) in enumerate(Gmap)); # Derived to original map
    S_tmp = Dict(i => Set([v]) for (i, v) in enumerate(Gmap));
    S_tmp[pseudo] = V_C;
    rev_S_tmp = Dict{Int, Int}();
    for (i, V) in S_tmp
        for v in V
            rev_S_tmp[v] = i;
        end
    end
    S_new = Dict(i => S[v] for (i, v) in enumerate(Gmap)); # Derived to original map
    S_new[pseudo] = union([S[v] for v in V_C]...); # Pseudonode map
    rev_S_new = Dict{Int, Int}(); # Original to derived map
    for (i, V) in S_new
        for v in V
            rev_S_new[v] = i;
        end
    end
    
    println(rev_S_tmp)
        
    
    # Create new tree T' with E(T') = E(T) \ E(C)
    V_T = union(T.A, T.B);
    safe = [v for v in V_T if rev_S_tmp[v] != pseudo];
    if isempty(safe)
        Tnew_graph = SimpleGraph(pseudo);
    else
        Tnew_graph, _ = induced_subgraph(T.graph, [v for v in V_T if rev_S_tmp[v] != pseudo]);
    end
    add_vertex!(Tnew_graph);
    Tnew_A = Set([rev_S_tmp[v] for v in T.A if rev_S_tmp[v] != pseudo]); # Convert original A to derived A
    Tnew_B = Set([rev_S_tmp[v] for v in T.B]); # Convert original B to derived B
    Tnew_root = rev_S_tmp[T.root];
    println(Tnew_A)
    
    # Copy M into Mnew and map to derived vertices
    Mnew = Set([UEdge(rev_S_new[e.src], rev_S_new[e.dst]) for e in M]);
    # Remove matches in C from M
    pop!(Mnew, UEdge(pseudo, pseudo));
    
    edges_to_C, C_inside, C_neighbors = cut(G, V_C);
    for (i, e) in enumerate(edges_to_C)
        n = C_neighbors[i]; # Get neighbor of C
        n_derived = first(rev_S_tmp[n]); # Derived index of neighbor
        add_edge!(Gnew, n_derived, pseudo); # Connect neighbor to the new pseudonode's index
        if has_edge(T.graph, e)
            add_edge!(Tnew_graph, n_derived, pseudo); # Connect neighbor edge in Tnew
        end
    end
    
    # Mnew = M \ E(C)
    
#     # Modify alternating tree
#     T.A = Tnew_A;
#     T.B = Tnew_B;
#     T.root = Tnew_root;
#     T.graph = Tnew_graph;
    println(Tnew_graph)
    Tnew = AltTree(Tnew_root, Tnew_A, Tnew_B, Mnew, Tnew_graph);
    
    if isa(Gnew, MetaGraph) || isa(Gnew, MetaDiGraph)
        positions = [get_prop(G, v, :position) for v in V_C];
        centroid = (sum(p[1] for p in positions)/length(V_C), sum(p[2] for p in positions)/length(V_C));
        place!(Gnew, pseudo, centroid); # Set pseudo node position to centroid
    end
    
    println(Tnew_graph)
    println("Old T: $(T)")
    println("New T: $(Tnew)");
    
    return Gnew, Mnew, Tnew, S_new, rev_S_new, S, S_tmp, rev_S_tmp, pseudo, V_C, C;
end;

# Prints a nice representation of an alternating tree
function Base.print(io::IO, T::AltTree)
    Base.print(io, "{root: $(T.root); A: {" * join(T.A, ", "), "}; B: {" * join(T.B, ", ") * "}; "
        * "E(T): $(Set(UEdge(e) for e in edges(T.graph)))}");
end;

In [ ]:
struct DerivedGraph{G}
    name::String # Name of derived graph
    graph::AbstractGraph # Underlying graph of derived graph
    M::Set{UEdge{Int}} # Set of derived matchings
    T::AltTree # Alternating tree used in Blossom algorithm
    D2O::Dict{Int, Set{Int}} # Map from derived to original indices
    O2D::Dict{Int, Int} # Map from original to derived indices (reverse map)
    c::Int # Index of blossom used to create derived graph
    C::Union{Set{UEdge{Int}}, Nothing} # Set of edges in blossom used to create graph
    function DerivedGraph(name::String, g::G, M::Set{UEdge{Int}}, T::AltTree=AltTree(g, M),
        c::Int=-1, C::Union{Set{UEdge{Int}}, Nothing}=nothing,
        D2O::Union{Dict{Int, Set{Int}}, Nothing}=nothing) where {G <: AbstractGraph}
        O2D = Dict{Int, Int}();
        if D2O == nothing
            D2O = Dict(v => Set([v]) for v in vertices(g)); # Populate S if empty
        end
        for (d, s) in D2O
            for v in s
                O2D[v] = d; # Construct reverse map
            end
        end
        return new{G}(name, g, T, D2O, O2D, c, C);
    end
end;

DerivedGraph(g::G, M::Set{UEdge{Int}}=Set{UEdge{Int}}(), T::AltTree=AltTree(g, M),
        c::Int=-1, C::Union{Set{UEdge{Int}}, Nothing}=nothing,
        D2O::Union{Dict{Int, Set{Int}}, Nothing}=nothing) where {G <: AbstractGraph} = DerivedGraph("G'",
            g, M, T, c, C, D2O);

In [28]:
# Prints a nice representation of derived graph data
function Base.print(io::IO, D::DerivedGraph)
    vertex_str = "V: {" * join(vertices(D.graph), ", ") * "}";
    edge_str = "E: {" * join([UEdge(e) for e in edges(D.graph)], ", ") * "}";
    M_str = "M: $(D.M)";
    T_str = "T: $(D.T)";
    D2O_str = "D2O:\n" * join(["\t$(d) => {" * join(D.D2O[d], ", ") * "}" for d in keys(D.D2O)], "\n");
    blossom_str = "Blossom: " * (D.C == nothing ? "none" : join(D.C, ",") * " => {$(D.c)}");
    Base.print(io, "$(D.name)[" * join([vertex_str, 
                edge_str, M_str, T_str, D2O_str, blossom_str], "\n") * " ]");
end;

In [29]:
g = SimpleGraph(3);

D = DerivedGraph(g);

println(D)

G'[V: {1, 2, 3}
E: {}
M: {}
T: {root: 2; A: {}; B: {2}; E(T): Set{UEdge}()}
D2O:
	2 => {2}
	3 => {3}
	1 => {1}
Blossom: none ]


In [ ]:
# New shrinking function
function shrink(D::DerivedGraph{G}) where {G <: AbstractGraph}
    if !in(v, D.T.B) || !in(w, D.T.B) || !has_edge(D.graph, v, w)
        return false;
    end
    
    # Get circuit C
    C_path = BFS(D.T.graph, v, w);
#     add_edge!(T.graph, v, w);
    C = connect([C_path..., v], UEdge{Int});
    C_edges = Set(C);
    V_C = Set(C_path);
    
    # Create new graph G' = G x C
    V_GP = [v for v in vertices(G) if !in(v, V_C)];
    Gnew, Gmap = induced_subgraph(G, V_GP);
    add_vertex!(Gnew);
    pseudo = nv(Gnew); # Create the pseudo node
    
    if isempty(S)
        S = Dict(v => Set([v]) for v in vertices(G));
    end
    
    # Generate maps
#     s = Dict(i => Set([v]) for (i, v) in enumerate(Gmap)); # Derived to original map
    S_tmp = Dict(i => Set([v]) for (i, v) in enumerate(Gmap));
    S_tmp[pseudo] = V_C;
    rev_S_tmp = Dict{Int, Int}();
    for (i, V) in S_tmp
        for v in V
            rev_S_tmp[v] = i;
        end
    end
    S_new = Dict(i => S[v] for (i, v) in enumerate(Gmap)); # Derived to original map
    S_new[pseudo] = union([S[v] for v in V_C]...); # Pseudonode map
    rev_S_new = Dict{Int, Int}(); # Original to derived map
    for (i, V) in S_new
        for v in V
            rev_S_new[v] = i;
        end
    end
    
    println(rev_S_tmp)
        
    
    # Create new tree T' with E(T') = E(T) \ E(C)
    V_T = union(T.A, T.B);
    safe = [v for v in V_T if rev_S_tmp[v] != pseudo];
    if isempty(safe)
        Tnew_graph = SimpleGraph(pseudo);
    else
        Tnew_graph, _ = induced_subgraph(T.graph, [v for v in V_T if rev_S_tmp[v] != pseudo]);
    end
    add_vertex!(Tnew_graph);
    Tnew_A = Set([rev_S_tmp[v] for v in T.A if rev_S_tmp[v] != pseudo]); # Convert original A to derived A
    Tnew_B = Set([rev_S_tmp[v] for v in T.B]); # Convert original B to derived B
    Tnew_root = rev_S_tmp[T.root];
    println(Tnew_A)
    
    # Copy M into Mnew and map to derived vertices
    Mnew = Set([UEdge(rev_S_new[e.src], rev_S_new[e.dst]) for e in M]);
    # Remove matches in C from M
    pop!(Mnew, UEdge(pseudo, pseudo));
    
    edges_to_C, C_inside, C_neighbors = cut(G, V_C);
    for (i, e) in enumerate(edges_to_C)
        n = C_neighbors[i]; # Get neighbor of C
        n_derived = first(rev_S_tmp[n]); # Derived index of neighbor
        add_edge!(Gnew, n_derived, pseudo); # Connect neighbor to the new pseudonode's index
        if has_edge(T.graph, e)
            add_edge!(Tnew_graph, n_derived, pseudo); # Connect neighbor edge in Tnew
        end
    end
    
    # Mnew = M \ E(C)
    
#     # Modify alternating tree
#     T.A = Tnew_A;
#     T.B = Tnew_B;
#     T.root = Tnew_root;
#     T.graph = Tnew_graph;
    println(Tnew_graph)
    Tnew = AltTree(Tnew_root, Tnew_A, Tnew_B, Mnew, Tnew_graph);
    
    if isa(Gnew, MetaGraph) || isa(Gnew, MetaDiGraph)
        positions = [get_prop(G, v, :position) for v in V_C];
        centroid = (sum(p[1] for p in positions)/length(V_C), sum(p[2] for p in positions)/length(V_C));
        place!(Gnew, pseudo, centroid); # Set pseudo node position to centroid
    end
    
    println(Tnew_graph)
    println("Old T: $(T)")
    println("New T: $(Tnew)");
    
    return Gnew, Mnew, Tnew, S_new, rev_S_new, S, S_tmp, rev_S_tmp, pseudo, V_C, C;
end;

In [ ]:
function perfect_matching_blossom(g::AbstractGraph, M_input::Set{UEdge{Int}}=Set{UEdge{Int}}(); 
        verbose::Bool=false, show_graphs::Bool=false)
    log(msg::Any) = verbose ? println(msg) : nothing # printer

    M = copy(M_input);
    G = copy(g);
    covered(M::Set{UEdge{Int}}) = union(Set(e.src for e in M), Set(e.dst for e in M));
    exposed(M::Set{UEdge{Int}}) = setdiff(Set(v for v in vertices(g)), covered(M));
    M_exposed = exposed(M);
    T = AltTree(G, M);
    S = Dict(v => Set([v]) for v in vertices(G)); # Master map
    S_old = copy(S);
    rev_S = Dict{Int, Int}();
    c = -1; # Initialize pseudo node to -1
    M_old = Set{UEdge{Int}}();
    M_output = copy(M_input);
    
    # Returns an extended M to original graph as output M_output
    function extend(M::Set{UEdge{Int}}, M_old::Set{UEdge{Int}}, S::Dict{Int, Set{Int}}, 
            S_old::Dict{Int, Set{Int}}, c::Int)
        println("M new: $(M)");
        println("M_old: $(M_old)")
        M_output = Set{UEdge{Int}}();
        for ue in M
            if !incident(c, ue)
                println(ue)
                s, d = first(S_old[ue.src]), first(S_old[ue.dst]);
                oe = UEdge(s, d); # Original edge
                push!(M_output, oe);
            end
        end
        for ue in M_old
            if in(ue.src, S[c]) && in(ue.dst, S[c])
                push!(M_output, ue);
#             elseif in(ue.src, S[c]) || in(ue.dst, S[c])
#                 push!(M_output, ue);
            end
        end
        println("M output: $(M_output)")
        return M_output;
    end
    
    log("M initialized to $(M)")
    log("=== Loop START ===")
    step = 1;
    while true
        exit = true;
        log("--- Step $(step) ---")
        for e in edges(G)
            v, w = e.src, e.dst;
            if in(v, T.B) || in(w, T.B)
                v, w = (in(v, T.B) ? (v, w) : (w, v)); # Reassign v to the vertex in B(T)
                if in(v, T.B) && !in(w, T.A)
                    log("v: $(v), w: $(w)")
                    exit = false;
                    if !in(w, T.B)
                        if in(w, M_exposed)
                            augment!(T, v, w);
                            log("Augmented M: $(M)");
                            M_output = extend(M, M_old, S, S_old, c);
                            M_exposed = exposed(M);
                            if isempty(M_exposed)
                                exit = true;
                            else
                                T = AltTree(G, M);
                                log("New T: $(T)")
                            end
                        else
                            extend!(T, v, w);
                            log("Extended T: $(T)")
                        end
                    else
                        M_old = M;
                        V_old = vertices(G);
#                         Gnew, Mnew, Tnew, S_new, S, S_tmp, rev_S_tmp, pseudo, V_C
                        G, M, T, S, S_old, S_tmp, rev_S_tmp, c, V_C = shrink(G, T, v, w, S);
                        log("Shrunk M', G', T");
                        log("M: $(M)")
                        log("M': $(M_old)");
                        log("T: $(T)");
                        log("C: " * join(V_C, ", "))
                        log("Old map:");
                        for v in V_old
                            log("S[$(v)]: " * join(S_old[v], ", "))
                        end
                        log("New map:");
                        for v in vertices(G)
                            log("S[$(v)]: " * join(S[v], ", "))
                        end
                    end
                    break
                end
            end
        end
        if exit
            log("No perfect matching")
            break
        end
        step += 1;
    end
    log("=== Loop END ===")
    log("Final M: $(M_output)")
    found = isempty(M_exposed);
    if found
        log("Perfect Matching M found...")
        log("M: $(M_output)")
        log("|M|: $(length(M))")
    end
    return M, found;
end;

In [ ]:
g = SimpleGraph(3)
add_edge!(g, 1, 2)
graphplot(g; names=vertices(g), edgecolors=1)

In [ ]:
# Example graph from final
E = Set{Edge{Int}}();

V_str = "abcdfghknpqm"; # vertex names
V_names = to_sym(collect(V_str)); # creates symbolic vertice array
index = Dict(name => i for (i, name) in enumerate(V_names)); # index map

push!(E, Edge(index[:a], index[:d]));
push!(E, Edge(index[:a], index[:c]));
push!(E, Edge(index[:c], index[:d]));
push!(E, Edge(index[:c], index[:h]));
push!(E, Edge(index[:d], index[:n]));
push!(E, Edge(index[:h], index[:n]));
push!(E, Edge(index[:h], index[:k]));
push!(E, Edge(index[:k], index[:n]));
push!(E, Edge(index[:h], index[:m]));
push!(E, Edge(index[:n], index[:p]));
push!(E, Edge(index[:p], index[:q]));
push!(E, Edge(index[:n], index[:q]));
push!(E, Edge(index[:n], index[:f]));
push!(E, Edge(index[:f], index[:g]));
push!(E, Edge(index[:f], index[:b]));
push!(E, Edge(index[:b], index[:g]));
push!(E, Edge(index[:q], index[:m]));
push!(E, Edge(index[:g], index[:q]));

mg = MetaGraph(SimpleGraphFromIterator(E));

place!(mg, index[:n], (0, 0));
place!(mg, index[:h], (-3, 0));
place!(mg, index[:q], (3, 0));
place!(mg, index[:k], (-1, -1));
place!(mg, index[:p], (1, -1));
place!(mg, index[:d], (-1, 2));
place!(mg, index[:c], (-2, 2));
place!(mg, index[:a], (-1.5, 3));
place!(mg, index[:f], (1, 2));
place!(mg, index[:g], (2, 2));
place!(mg, index[:b], (1.5, 3));
place!(mg, index[:m], (0, -3));

M = Set{UEdge{Int}}();
push!(M, UEdge(index[:a], index[:c]));
push!(M, UEdge(index[:h], index[:n]));
push!(M, UEdge(index[:b], index[:g]));
push!(M, UEdge(index[:p], index[:q]));

T = AltTree(mg, M);
T.root = index[:d];
T.A = Set{Int}([index[:a]]);
T.B = Set{Int}([index[:d], index[:c]]);

add_edge!(T.graph, index[:d], index[:a]);
add_edge!(T.graph, index[:a], index[:c]);

function aug_ew(T; M_weight=3, T_weight=1.2, default=0.1)
    function ew(s, d, w)
        e = Edge(s, d);
        ue = UEdge(s, d);
        if in(ue, T.M)
            return M_weight;
        elseif has_edge(T.graph, e)
            return T_weight;
        else
            return default;
        end
    end
    return ew;
end

function plot_derived(mg::MetaGraph, T::AltTree,
        S::Dict{Int, Set{Int}}=Dict(v => Set([v]) for v in vertices(mg)); 
        T_color::Union{Color, Int}=RGB(0, 1, 0), root_color::Union{Color, Int}=RGB(1, 1, 0),
        default_color::Union{Color, Int}=1, curvature=0, nodesize=0.13, names=vertices(mg),
        A_shape::Symbol=:rect, B_shape::Symbol=:circle, default_shape::Symbol=:hexagon, args...)
    in_T(v) = (in(v, T.A) || in(v, T.B));
    node_colors = [in_T(v) ? (v == T.root ? root_color : T_color) : default_color for v in vertices(mg)];
    node_shapes = [in(v, T.A) ? A_shape : (in(v, T.B) ? B_shape : default_shape) for v in vertices(mg)];
    node_scales = [length(S[v]) for v in vertices(mg)];
    function edge_colors(s, d, w)
        e = Edge(s, d)
        return has_edge(T.graph, e) ? T_color : :black;
    end
    return plot_fixed(mg; names=names, curvature=curvature, nodesize=nodesize,
        nodecolor=node_colors, nodeshape=node_shapes, edgecolor=edge_colors,
        nodeweights=node_scales, ew=emphasize([Edge(ue) for ue in T.M]), args...);
end

# Returns an extended M to original graph as output M_output
function extend(M_prime::Set{UEdge{Int}}, G_prime::AbstractGraph, 
        M_old::Set{UEdge{Int}}, S::Dict{Int, Set{Int}}, 
        rev_S::Dict{Int, Int}, S_old::Dict{Int, Set{Int}}, c::Int, C::Vector{UEdge{Int}})
    println("M': $(M_prime)");
#     println("Old M': $(M_prime_old)")
    println("Old M: $(M_old)")
    M_new = Set{UEdge{Int}}();
    for ue in M_prime
        if incident(c, ue)
            w = -1
            n = (ue.src == c ? ue.dst : ue.src); # Get connected neighbor of pseudo node
            on = first(S[n]); # original neighbor index
            println("n: $(n)")
            println("Original n: $(on)")
            for v in S[c]
                println("v: $(v)")
                if has_edge(G_prime, on, v)
                    w = v;
                    break
                end
            end
            # get the original node in pseudonode connected to the n
            println("w: $(w)");
            _C = filter((e)->!incident(w, e), C); # get the edges in C with w removed
            union!(M_new, Set(_C[1:2:end])); # Construct new extended matching
            
            push!(M_new, UEdge(on, w));
        else
            os, od = first(S[ue.src]), first(S[ue.dst]);
            push!(M_new, UEdge(os, od));
        end
    end
#     for ue in M_old
#         if in(ue.src, S[c]) && in(ue.dst, S[c])
#             push!(M_new, ue);
# #             elseif in(ue.src, S[c]) || in(ue.dst, S[c])
# #                 push!(M_output, ue);
# #         elseif in(ue.src, S[c]) || in(ue.dst, S[c]) # the edge covers the pseudonode
# #             # get the original node in pseudonode connected to the outside
# #             w = (rev_S[ue.src] == c ? ue.src : ue.dst);
# #             _C = filter((e)->!incident(w, e), C); # get the edges in C with w removed
# #             union!(M_new, Set(_C[1:2:end])); # Construct new extended matching
# #         end
# #         elseif in(ue.src, S[c]) || in(ue.dst, S[c]) # the edge covers the pseudonode
            
#         end
#     end
    println("New M: $(M_new)")
    return M_new;
end

show_plot(plot_derived(mg, T); title="Original graph");
println("Original M: $(M)")

Gnew, Mnew, Tnew, S, rev_S, S_old, S_tmp, rev_S_tmp, c, V_C, C = shrink(mg, T, 3, 4);

println("C: [" * join(C, ", ") * "]")
println("Original M after shrinking: $(M)")
show_plot(plot_derived(Gnew, Tnew, S); title="Derived graph");

println("---")
println("New M': $(Mnew)")
for (d, s) in S
    println("S[$(d)]: {" * join(S[d], ",") * "}")
end

println("M': $(Mnew), T: $(Tnew)")
extend!(Tnew, c, 4);
println("Extended T: $(Tnew)")


show_plot(plot_derived(Gnew, Tnew, S); title="Derived graph");

Mnew_old = copy(Mnew);
augment!(Tnew, 6, 5);
println("Augmented M': $(Mnew)")
Tnew = AltTree(Gnew, Mnew);
show_plot(plot_derived(Gnew, Tnew, S); title="Derived graph");

M = extend(Mnew, mg, T.M, S, rev_S, S_old, c, C)

println("Actual M: $(M)")


T = AltTree(mg, M);
show_plot(plot_derived(mg, T); title="Underived graph");
# show_plot(plot_fixed(Gnew; names=vertices(Gnew), 
#         ew=aug_ew(T),
#         curvature=0, nodesize=0.13); title="Derived graph");

# show_plot(plot_fixed(Gnew; names=[v == c ? "C'" : first(S_new[v]) for v in vertices(Gnew)], 
#         ew=emphasize([Edge(ue) for ue in Mnew]),
#         curvature=0, nodesize=0.13));

# perfect_matching_blossom(mg; verbose=true, show_graphs=true);


In [ ]:
# Example graph from slide 42 of Optimal Matchings
E = Set{Edge{Int}}();

push!(E, Edge(1, 2));
push!(E, Edge(1, 10));
push!(E, Edge(1, 5));
push!(E, Edge(2, 3));
push!(E, Edge(2, 4));
push!(E, Edge(3, 4));
push!(E, Edge(4, 13));
push!(E, Edge(5, 6));
push!(E, Edge(6, 7));
push!(E, Edge(6, 8));
push!(E, Edge(7, 8));
push!(E, Edge(13, 8));
push!(E, Edge(13, 9));
push!(E, Edge(10, 11));
push!(E, Edge(9, 11));
push!(E, Edge(8, 12));
push!(E, Edge(8, 11));
push!(E, Edge(11, 12));
push!(E, Edge(1, 14));
push!(E, Edge(14, 15));
push!(E, Edge(11, 15));
push!(E, Edge(1, 16));
push!(E, Edge(16, 11));
push!(E, Edge(5, 7));

mg = MetaGraph(SimpleGraphFromIterator(E));

place!(mg, 1, (0, 0));
place!(mg, 2, (-1, -2));
place!(mg, 3, (-2, -3));
place!(mg, 4, (0, -3));
place!(mg, 5, (3, -2));
place!(mg, 6, (2, -3));
place!(mg, 7, (4, -3));
place!(mg, 8, (3, -4));
place!(mg, 9, (-0.5, -4.5));
place!(mg, 10, (-3, -3.5));
place!(mg, 11, (-0.5, -8));
place!(mg, 12, (3.5, -5));
place!(mg, 13, (1, -3.75));
place!(mg, 14, (-5, -3));
place!(mg, 15, (-5, -5));
place!(mg, 16, (8, -3));

M = Set{UEdge{Int}}([UEdge(14, 15), UEdge(10, 11), UEdge(1, 5),
        UEdge(2, 3), UEdge(9, 13), UEdge(6, 7), UEdge(8, 12)]);

T = AltTree(mg, M);
T.root = 4;
T.A = Set{Int}([2, 13, 11, 1, 6, 8]);
T.B = Set{Int}([4, 3, 9, 10, 5, 7, 12]);

add_edge!(T.graph, 4, 2);
add_edge!(T.graph, 2, 3);
add_edge!(T.graph, 4, 13);
add_edge!(T.graph, 13, 9);
add_edge!(T.graph, 9, 11);
add_edge!(T.graph, 11, 10);
add_edge!(T.graph, 10, 1);
add_edge!(T.graph, 1, 5);
add_edge!(T.graph, 5, 6);
add_edge!(T.graph, 6, 7);
add_edge!(T.graph, 7, 8);
add_edge!(T.graph, 8, 12);

show_plot(plot_fixed(mg; curvature=0, nodesize=0.13, names=collect(vertices(mg)),
        ew=emphasize([Edge(ue) for ue in M])));

Gnew, Mnew, Tnew, S_new, S, S_tmp, rev_S_tmp, c, V_C = shrink(mg, T, 7, 5);

show_plot(plot_fixed(Gnew; names=[v == c ? "C'" : first(S_new[v]) for v in vertices(Gnew)], 
        ew=emphasize([Edge(ue) for ue in Mnew]),
        curvature=0, nodesize=0.13));

# perfect_matching_blossom(mg; verbose=true, show_graphs=true);


In [ ]:
function perfect_matching_bipartite(g::AbstractGraph; verbose::Bool=false, show_graphs::Bool=false)
    log(msg::Any) = verbose ? println(msg) : nothing # printer

    M = Set{UEdge{Int}}();
    covered(M::Set{UEdge{Int}}) = union(Set(e.src for e in M), Set(e.dst for e in M));
    exposed(M::Set{UEdge{Int}}) = setdiff(Set(v for v in vertices(g)), covered(M));
    M_exposed = exposed(M);
    T = AltTree(g, M);
    
    log("M initialized to $(M)")
    log("=== Loop START ===")
    step = 1;
    while true
        exit = true;
        log("--- Step $(step) ---")
        for e in edges(g)
            v, w = e.src, e.dst;
            if in(v, T.B) || in(w, T.B)
                v, w = (in(v, T.B) ? (v, w) : (w, v)); # Reassign v to the vertex in B(T)
                if !in(w, union(T.A, T.B))
                    log("v: $(v), w: $(w)")
                    exit = false;
                    if in(w, M_exposed)
                        augment!(T, v, w);
                        log("Augmented M: $(M)")
                        M_exposed = exposed(M);
                        if isempty(M_exposed)
                            exit = true;
                        else
                            T = AltTree(g, M);
                            log("New T: $(T)")
                        end
                    else
                        extend!(T, v, w);
                        log("Extended T: $(T)")
                    end
                    break
                end
            end
        end
        if exit
            log("No perfect matching")
            break
        end
        step += 1;
    end
    log("=== Loop END ===")
    log("Final M: $(M)")
    found = isempty(M_exposed);
    if found
        log("Perfect Matching M found...")
        log("M: $(M)")
        log("|M|: $(length(M))")
    end
    return M, found;
end;

In [ ]:
n = 7
g = SimpleGraph(n);
for (i, j) in zip(1:n-1, 2:n)
    add_edge!(g, i, j);
end
perfect_matching_bipartite(g; verbose=true, show_graphs=true)
# T = AltTree(g, 2);
# M = Set{UEdge{Int}}([UEdge(3, 4)]);
# println(T)
# println(M)
# extend!(M, T, 2, 3)
# println(T)
# println(M)
# augment!(M, T, 4, 5)
# println(T)
# println(M)

In [ ]:
P_str = "abcdefghij"; # Specify P here as a string
Q_str = uppercase(P_str); # Specify Q here as a string

P, Q = to_sym(collect(P_str)), to_sym(collect(Q_str)); # creates arrays for P and Q

L = Dict();
L[:a] = [:A, :C, :D];
L[:b] = [:A, :B, :H];
L[:c] = [:A, :C];
L[:d] = [:C, :D];
L[:e] = [:C, :D];
L[:f] = [:B, :E, :G];
L[:g] = [:E, :F, :J];
L[:h] = [:E, :F, :G, :H, :J];
L[:i] = [:I];
L[:j] = [:D, :I];

# Msize, M = max_matching_bipartite(P, Q, L; verbose=false, show_graphs=false, nodesize=0.25);

# println("Expected |M|: $(Msize)");

mg = build_bipartite(P, Q, L);

perfect_matching_bipartite(mg; verbose=true, show_graphs=true)

## TSP
- [Christofide's Algorithm](#Christofide's-Algorithm)
- [Christofide's Input](#Christofide's-Input)

### Christofide's Algorithm

In [ ]:
function Christofides(mg::MetaGraph;
        show_graphs::Bool=false, verbose::Bool=false, 
        names::Vector{S}=Vector(1:nv(mg)), args...) where {S <: Union{String, Symbol, Int}}
    
    ename(e::T) where {T <: AbstractEdge} = "$(names[e.src]) <=> $(names[e.dst])";
    log(msg::Any) = verbose ? println(msg) : nothing # printer
    
    # Dictionary of weights
    wd = Dict((e.src, e.dst) => get_prop(mg, e, :weight) for e in edges(mg));
    for e in edges(mg)
        re = reverse(e);
        wd[re.src, re.dst] = wd[e.src, e.dst]; # add reverse edges too
    end
    
    show_plot(plot_fixed(mg; names=names, el=wd, curvature=0.01, args...); show=show_graphs, title="Input graph G");
    
    mst_edges = kruskal_mst(mg); # get MST
    T, Tmap = induced_subgraph(mg, mst_edges);
    log("c(T): " * sum_str(wts(mst_edges, wd)));
    show_plot(plot_fixed(mg; el=Dict((e.src, e.dst)=>wd[e.src, e.dst] for e in mst_edges), 
        ew=emphasize(mst_edges; in_weight=3, out_weight=0.2), curvature=0, names=names, args...);
        show=show_graphs, title="MST T");
    
    W = S[];
    for v in vertices(T)
        deg = degree(T, v);
        log("deg($(names[Tmap[v]])) = $(deg)")
        if deg % 2 == 1
            push!(W, Tmap[v])
        end
    end
    log("W (odd-degree nodes in T): " * join(W, ", "))
    
    sg, Wmap = induced_subgraph(mg, W);
    match = minimum_weight_perfect_matching(SimpleGraph(sg), 
        Dict(e => wd[Wmap[e.src], Wmap[e.dst]] for e in edges(sg)));
    el = Dict([(e.src, e.dst)=>wd[Wmap[e.src], Wmap[e.dst]] for e in edges(sg)]...);
    for e in edges(sg)
        el[e.dst, e.src] = wd[Wmap[e.src], Wmap[e.dst]]; # add reverse edges
    end
    inv_Wmap = Dict(v => i for (i, v) in enumerate(Wmap)); # Inverse W map
    M = unique([Edge(Wmap[v], Wmap[match.mate[v]]) for v in vertices(sg)]);
    log("Matching M:\n" * join(["$(names[e.src]) <=> $(names[e.dst])" for e in M], "\n"))
    log("c(M): " * sum_str(wts(M, wd)))
    show_plot(plot_fixed(sg; ew=emphasize(map((e)->Edge(inv_Wmap[e.src], inv_Wmap[e.dst]), M); out_weight=0.2),
            el=el, pos=[get_prop(mg, Wmap[v], :position) for v in vertices(sg)],
            curvature=0, names=Wmap, args...); show=show_graphs, title="Min-cost Matching M of G[W]");
    
    C = unique([mst_edges..., M...]); # Euler circuit with M + mst
    cg, Cmap = induced_subgraph(mg, C);
    show_plot(plot_fixed(mg; ew=emphasize(C; out_weight=0.2), el=wd, curvature=0,
            names=names, args...); show=show_graphs, title="T + M")
    
    log("Combined T + M:")
    log(join(["$(ename(e))" for e in C], "\n"))
    return C;
end;

### Christofide's Input

In [ ]:
mg = MetaGraph(SimpleGraph(8));

wd = Dict{Tuple{Int, Int}, Float64}();

positions = Dict();
positions[1] = (0,0); positions[2] = (2,0); positions[3] = (4,0);
positions[4] = (1,-2); positions[5] = (3,-2);
positions[6] = (-0.5,-3); positions[7] = (1,-3.5); positions[8] = (4,-3);
for (v, pos) in positions
    x, y = pos;
    place!(mg, v, (x, y));
end

connect!(mg, 1, 2, 11; wd=wd);
connect!(mg, 2, 3, 11; wd=wd);
connect!(mg, 1, 4, 4; wd=wd);
connect!(mg, 1, 6, 7; wd=wd);
connect!(mg, 2, 4, 7; wd=wd);
connect!(mg, 2, 5, 6; wd=wd);
connect!(mg, 3, 5, 5; wd=wd);
connect!(mg, 3, 8, 6; wd=wd);
connect!(mg, 4, 5, 1; wd=wd);
connect!(mg, 4, 6, 3; wd=wd);
connect!(mg, 4, 7, 9; wd=wd);
connect!(mg, 5, 8, 2; wd=wd);
connect!(mg, 6, 7, 7; wd=wd);
connect!(mg, 7, 8, 11; wd=wd);

In [ ]:
C = Christofides(mg; show_graphs=true, verbose=true, elb=false, elo=0.015, curvature=0);

In [ ]:
tour = C[:];
remove!(Edge(4, 6), tour);
push!(tour, Edge(6, 1));

remove!(Edge(2, 5), tour);
remove!(Edge(4, 5), tour);
push!(tour, Edge(4, 2));

In [ ]:
# tour = C[:];
# remove!(Edge(4, 6), tour);
# push!(tour, Edge(6, 1));

# remove!(Edge(2, 5), tour);
# remove!(Edge(4, 5), tour);
# push!(tour, Edge(4, 2));

println("Tour: ")
nodes = tour[:];
path = [1];
while !isempty(nodes)
    curr = last(path);
    next = remove!(Edge(1, 2), nodes; condition=(e)->incident(curr, e));
    if isempty(next)
        push!(path, 1)
        break;
    end
    edge = next[1];
    curr = (edge.src == curr ? edge.dst : edge.src)
    push!(path, curr)
end
push!(path, 1)
println(join(path, " <-> "))

tour = connect(path);

println("Cost of Christofide's Tour: " * sum_str(wts(tour, wd)))

plot_fixed(mg; ew=emphasize(tour; out_weight=0.2), el=wd, curvature=0, elb=false, elo=0.025)